In [75]:
# Importing librariesd and loading openAI API key

import pandas as pd
import os
import openai
import re


openai.api_key = ''

In [2]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 1.2 MB/s eta 0:00:00


In [5]:
# Function to get the response from ChatGPT through API key

def get_completion_from_messages(messages,
                                 model="gpt-3.5-turbo",
                                 temperature=0,
                                 max_tokens=1000):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )
    return response.choices[0].message["content"]

In [6]:
df1=pd.read_csv('https://raw.githubusercontent.com/atreish/yukatzbot/main/tests/questions_answers_cleaned_programs.csv')


In [7]:
df2=pd.read_csv('https://raw.githubusercontent.com/atreish/yukatzbot/main/tests/questions_answers_cleaned_faculty.csv')


In [8]:
df=df1.append(df2).reset_index(drop=True)




<ipython-input-8-5fe87a33e7b8>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df1.append(df2).reset_index(drop=True)


In [ ]:
# Reading the question and answers data extracted earlier from chatGPT

df = pd.read_csv("https://raw.githubusercontent.com/atreish/yukatzbot/main/tests/questions_answers_cleaned.csv")
df

In [66]:
# Function to extract entities from question
def ent_extract(i):
    """
    This function takes question statement from the dataframe of question and answers and return entities
    """
    messages =  [
        {'role':'system',
         'content':"""You are a helpful assistant who performs entity extraction. Your answer should be the primary and  one secondary entity , with nothing else.
         Primary entity should be only program title - or facaulty name like """},
        {'role':'user',
         'content':"""Extract primary and secondary entities from this question statement - {}""".format(i)},
    ]
    response = get_completion_from_messages(messages)
    return response


In the following code blocks, we are extracting entities for 100 question statements at a time due to avoid the failure of API calls

In [67]:
entities=[]

In [68]:
entities = [ent_extract(i) for i in df[:]['question']]

In [70]:
entities

["Primary Entity: Master's degree program in artificial intelligence at Yeshiva University\nSecondary Entity: Applications",
 "Primary Entity: Master's degree program in artificial intelligence at Yeshiva University\nSecondary Entity: Skills and knowledge",
 "Primary entity: Master's degree program in artificial intelligence at Yeshiva University\nSecondary entity: Job opportunities",
 "Primary entity: Yeshiva University\nSecondary entity: master's degree in artificial intelligence",
 "Primary entity: Master's degree program in artificial intelligence at Yeshiva University\nSecondary entity: Internships and practical training opportunities",
 'Primary entity: M.S. in Biotechnology Management and Entrepreneurship program\nSecondary entity: goal',
 'Primary entity: Biotech Fellows program\nSecondary entity: students',
 'Primary entity: B.A./M.S. Pathways option\nSecondary entity: Yeshiva College and Stern College for Women',
 "Primary entity: Master's in Biotechnology Management and Entr

In [71]:
df['entities'] = entities


In [72]:
df

,question,answer,entities
0,What are some of the applications that student...,Students in the program can work on applicati...,Primary Entity: Master's degree program in art...
1,What are some of the skills and knowledge that...,Students will gain skills and knowledge in AI...,Primary Entity: Master's degree program in art...
2,What are some of the job opportunities availab...,Graduates can pursue careers in research and ...,Primary entity: Master's degree program in art...
3,Can undergraduate students at Yeshiva Universi...,"Yes, through the B.A./M.S. Pathways option, u...",Primary entity: Yeshiva University\nSecondary ...
4,Are there opportunities for internships and pr...,"Yes, students have the opportunity to gain in...",Primary entity: Master's degree program in art...
...,...,...,...
89,Who is Marissa Barrera?,Marissa Barrera is a faculty member of the spe...,Primary entity: Marissa Barrera\nSecondary ent...
90,Who is Andrew Christler?,Andrew Christler is a faculty member of the sp...,Primary entity: Andrew Christler\nSecondary en...
91,Who is Troy Clifford Dargin?,Troy Clifford Dargin is a faculty member of th...,Primary entity: Troy Clifford Dargin\nSecondar...
92,Who is Michaela Medved?,Michaela Medved is a faculty member of the spe...,Primary entity: Michaela Medved\nSecondary ent...


In [129]:
primary_entities=[]
secondary_entities=[]

for i in range(len(df)):
  samples=df['entities'][i].split('Secondary')
  primary_entities.append(samples[0])
  if len(samples)>1:
      secondary_entities.append(samples[1])
  else:
      secondary_entities.append('')






In [130]:
primary_entities

["Primary Entity: Master's degree program in artificial intelligence at Yeshiva University\n",
 "Primary Entity: Master's degree program in artificial intelligence at Yeshiva University\n",
 "Primary entity: Master's degree program in artificial intelligence at Yeshiva University\n",
 'Primary entity: Yeshiva University\n',
 "Primary entity: Master's degree program in artificial intelligence at Yeshiva University\n",
 'Primary entity: M.S. in Biotechnology Management and Entrepreneurship program\n',
 'Primary entity: Biotech Fellows program\n',
 'Primary entity: B.A./M.S. Pathways option\n',
 "Primary entity: Master's in Biotechnology Management and Entrepreneurship program\n",
 "Primary entity: Master's in Biotechnology Management and Entrepreneurship program\n",
 "Primary entity: Master's in cybersecurity program at the Katz School\n",
 "Primary entity: Master's in cybersecurity program at the Katz School\n",
 "Primary entity: Master's in cybersecurity program at the Katz School\n",


In [163]:
def preprocessin(entities):
  entities=list(map(lambda x: re.sub('(Primary|Secondary)','',x).strip(),  entities))
  entities=list(map(lambda x: re.sub('Entity','',x),  entities))
  entities=list(map(lambda x: re.sub(r'\s+at Yeshiva University','',x),  entities))
  entities=list(map(lambda x: re.sub(r'\s+at the katz school','',x),  entities))
  entities=list(map(lambda x: re.sub(r'(\s+program\s*|\s+in\s+)',' ',x),  entities))
  entities=list(map(lambda x: re.sub(r'(\s+and\s+)',' ',x),  entities))


  entities=list(map(lambda x: re.sub('(entity|entities)','',x),  entities))
  entities=list(map(lambda x: re.sub('(:|\')','',x),  entities))
  entities = list(map(lambda x: re.sub(r'(\\n|\\n=)', '', x, flags=re.MULTILINE), entities))
  entities = list(map(lambda x: re.sub(r'(masters degree|masters)', 'm.s.', x, flags=re.MULTILINE), entities))
  entities = list(map(lambda x: re.sub(r'(/|\.)', '', x, flags=re.MULTILINE), entities))
  entities = list(map(lambda x: re.sub(r'(/|\.)', '', x, flags=re.MULTILINE), entities))



  entities = list(map(lambda x: x.strip().lower(), entities))

  return entities



In [172]:
secondary_entities=preprocessin(secondary_entities)


In [173]:
secondary_entities

['applications',
 'skills knowledge',
 'job opportunities',
 'ms artificial intelligence',
 'internships practical training opportunities',
 'goal',
 'students',
 'yeshiva college stern college for women',
 'internships',
 'stem-approved',
 'skills',
 'real-world challenges',
 'research industry projects',
 'katz school',
 'internships',
 'skills',
 'students',
 'yeshiva college stern college for women',
 'internships',
 'stem-approved degree',
 'skills',
 'completion time',
 'strategies',
 'internships',
 'international students',
 'career prospects for graduates',
 'yeshiva university',
 'yeshiva university',
 'bama pathway',
 'yeshiva university',
 'yeshiva university',
 'yeshiva university',
 'career opportunities',
 'internship opportunities for phd students',
 'yeshiva university',
 'katz school',
 'future careers',
 'other s the new york metropolitan area',
 'inter-professional learning',
 'career prospects',
 'healthcare field',
 'medical training',
 'practice locations',
 'res

In [164]:
primary_entities=preprocessin(primary_entities)


In [165]:
primary_entities

['ms artificial intelligence',
 'ms artificial intelligence',
 'ms artificial intelligence',
 'yeshiva university',
 'ms artificial intelligence',
 'ms biotechnology management entrepreneurship',
 'biotech fellows',
 'bams pathways option',
 'ms biotechnology management entrepreneurship',
 'ms biotechnology management entrepreneurship',
 'ms cybersecurity',
 'ms cybersecurity',
 'ms cybersecurity',
 'ms cybersecurity',
 'ms cybersecurity',
 'ms data analytics visualization',
 'ms data analytics visualization',
 'bams pathways option the ms data analytics visualization',
 'ms data analytics visualization',
 'ms data analytics visualization',
 'ms digital marketing media',
 'ms digital marketing media',
 'ms digital marketing media',
 'ms digital marketing media',
 'ms digital marketing media',
 'ma mathematics',
 'ma mathematics',
 'ma mathematics',
 'yeshiva university mathematics',
 'ma mathematics',
 'phd mathematical sciences',
 'phd mathematics',
 'phd mathematics',
 'mathematics',

In [182]:
df['primary_entities']=primary_entities
df['secondary_entities']=secondary_entities


In [176]:
del df['entities']


KeyError: ignored

In [168]:
df

,question,answer,primary_entities,secondary_entities
0,What are some of the applications that student...,Students in the program can work on applicati...,ms artificial intelligence,applications
1,What are some of the skills and knowledge that...,Students will gain skills and knowledge in AI...,ms artificial intelligence,skills and knowledge
2,What are some of the job opportunities availab...,Graduates can pursue careers in research and ...,ms artificial intelligence,job opportunities
3,Can undergraduate students at Yeshiva Universi...,"Yes, through the B.A./M.S. Pathways option, u...",yeshiva university,m.s. artificial intelligence
4,Are there opportunities for internships and pr...,"Yes, students have the opportunity to gain in...",ms artificial intelligence,internships and practical training opportunities
...,...,...,...,...
89,Who is Marissa Barrera?,Marissa Barrera is a faculty member of the spe...,marissa barrera,none
90,Who is Andrew Christler?,Andrew Christler is a faculty member of the sp...,andrew christler,none
91,Who is Troy Clifford Dargin?,Troy Clifford Dargin is a faculty member of th...,troy clifford dargin,none
92,Who is Michaela Medved?,Michaela Medved is a faculty member of the spe...,michaela medved,none


In [183]:
df=df.replace('none','')
#df=df.replace('faculty','')

In [184]:
df

,question,answer,primary_entities,secondary_entities,unique_intents
0,What are some of the applications that student...,Students in the program can work on applicati...,ms artificial intelligence,applications,ms_artificial_intelligence_applications
1,What are some of the skills and knowledge that...,Students will gain skills and knowledge in AI...,ms artificial intelligence,skills knowledge,ms_artificial_intelligence_skills_knowledge
2,What are some of the job opportunities availab...,Graduates can pursue careers in research and ...,ms artificial intelligence,job opportunities,ms_artificial_intelligence_job_opportunities
3,Can undergraduate students at Yeshiva Universi...,"Yes, through the B.A./M.S. Pathways option, u...",yeshiva university,ms artificial intelligence,yeshiva_university_ms_artificial_intelligence
4,Are there opportunities for internships and pr...,"Yes, students have the opportunity to gain in...",ms artificial intelligence,internships practical training opportunities,ms_artificial_intelligence_internships_practic...
...,...,...,...,...,...
89,Who is Marissa Barrera?,Marissa Barrera is a faculty member of the spe...,marissa barrera,,marissa_barrera_
90,Who is Andrew Christler?,Andrew Christler is a faculty member of the sp...,andrew christler,,andrew_christler_
91,Who is Troy Clifford Dargin?,Troy Clifford Dargin is a faculty member of th...,troy clifford dargin,,troy_clifford_dargin_
92,Who is Michaela Medved?,Michaela Medved is a faculty member of the spe...,michaela medved,,michaela_medved_


In [187]:
df['unique_intents']=df['primary_entities']+' '+df['secondary_entities']
df


,question,answer,primary_entities,secondary_entities,unique_intents
0,What are some of the applications that student...,Students in the program can work on applicati...,ms artificial intelligence,applications,ms artificial intelligence applications
1,What are some of the skills and knowledge that...,Students will gain skills and knowledge in AI...,ms artificial intelligence,skills knowledge,ms artificial intelligence skills knowledge
2,What are some of the job opportunities availab...,Graduates can pursue careers in research and ...,ms artificial intelligence,job opportunities,ms artificial intelligence job opportunities
3,Can undergraduate students at Yeshiva Universi...,"Yes, through the B.A./M.S. Pathways option, u...",yeshiva university,ms artificial intelligence,yeshiva university ms artificial intelligence
4,Are there opportunities for internships and pr...,"Yes, students have the opportunity to gain in...",ms artificial intelligence,internships practical training opportunities,ms artificial intelligence internships practic...
...,...,...,...,...,...
89,Who is Marissa Barrera?,Marissa Barrera is a faculty member of the spe...,marissa barrera,,marissa barrera
90,Who is Andrew Christler?,Andrew Christler is a faculty member of the sp...,andrew christler,,andrew christler
91,Who is Troy Clifford Dargin?,Troy Clifford Dargin is a faculty member of th...,troy clifford dargin,,troy clifford dargin
92,Who is Michaela Medved?,Michaela Medved is a faculty member of the spe...,michaela medved,,michaela medved


In [188]:
df['unique_intents']=df['unique_intents'].str.replace(' ','_')
df


,question,answer,primary_entities,secondary_entities,unique_intents
0,What are some of the applications that student...,Students in the program can work on applicati...,ms artificial intelligence,applications,ms_artificial_intelligence_applications
1,What are some of the skills and knowledge that...,Students will gain skills and knowledge in AI...,ms artificial intelligence,skills knowledge,ms_artificial_intelligence_skills_knowledge
2,What are some of the job opportunities availab...,Graduates can pursue careers in research and ...,ms artificial intelligence,job opportunities,ms_artificial_intelligence_job_opportunities
3,Can undergraduate students at Yeshiva Universi...,"Yes, through the B.A./M.S. Pathways option, u...",yeshiva university,ms artificial intelligence,yeshiva_university_ms_artificial_intelligence
4,Are there opportunities for internships and pr...,"Yes, students have the opportunity to gain in...",ms artificial intelligence,internships practical training opportunities,ms_artificial_intelligence_internships_practic...
...,...,...,...,...,...
89,Who is Marissa Barrera?,Marissa Barrera is a faculty member of the spe...,marissa barrera,,marissa_barrera_
90,Who is Andrew Christler?,Andrew Christler is a faculty member of the sp...,andrew christler,,andrew_christler_
91,Who is Troy Clifford Dargin?,Troy Clifford Dargin is a faculty member of th...,troy clifford dargin,,troy_clifford_dargin_
92,Who is Michaela Medved?,Michaela Medved is a faculty member of the spe...,michaela medved,,michaela_medved_


In [189]:
df['unique_intents']=df['unique_intents'].apply(lambda x: re.sub('_$','',x))
df

df


,question,answer,primary_entities,secondary_entities,unique_intents
0,What are some of the applications that student...,Students in the program can work on applicati...,ms artificial intelligence,applications,ms_artificial_intelligence_applications
1,What are some of the skills and knowledge that...,Students will gain skills and knowledge in AI...,ms artificial intelligence,skills knowledge,ms_artificial_intelligence_skills_knowledge
2,What are some of the job opportunities availab...,Graduates can pursue careers in research and ...,ms artificial intelligence,job opportunities,ms_artificial_intelligence_job_opportunities
3,Can undergraduate students at Yeshiva Universi...,"Yes, through the B.A./M.S. Pathways option, u...",yeshiva university,ms artificial intelligence,yeshiva_university_ms_artificial_intelligence
4,Are there opportunities for internships and pr...,"Yes, students have the opportunity to gain in...",ms artificial intelligence,internships practical training opportunities,ms_artificial_intelligence_internships_practic...
...,...,...,...,...,...
89,Who is Marissa Barrera?,Marissa Barrera is a faculty member of the spe...,marissa barrera,,marissa_barrera
90,Who is Andrew Christler?,Andrew Christler is a faculty member of the sp...,andrew christler,,andrew_christler
91,Who is Troy Clifford Dargin?,Troy Clifford Dargin is a faculty member of th...,troy clifford dargin,,troy_clifford_dargin
92,Who is Michaela Medved?,Michaela Medved is a faculty member of the spe...,michaela medved,,michaela_medved


In [190]:
df[
    'answer'
] = df[
    'answer'
].apply(lambda x: x+'"')


In [191]:
df[
    'answer'
] = df[
    'answer'
].apply(lambda x: '"'+x)


In [192]:
df

,question,answer,primary_entities,secondary_entities,unique_intents
0,What are some of the applications that student...,""" Students in the program can work on applicat...",ms artificial intelligence,applications,ms_artificial_intelligence_applications
1,What are some of the skills and knowledge that...,""" Students will gain skills and knowledge in A...",ms artificial intelligence,skills knowledge,ms_artificial_intelligence_skills_knowledge
2,What are some of the job opportunities availab...,""" Graduates can pursue careers in research and...",ms artificial intelligence,job opportunities,ms_artificial_intelligence_job_opportunities
3,Can undergraduate students at Yeshiva Universi...,""" Yes, through the B.A./M.S. Pathways option, ...",yeshiva university,ms artificial intelligence,yeshiva_university_ms_artificial_intelligence
4,Are there opportunities for internships and pr...,""" Yes, students have the opportunity to gain i...",ms artificial intelligence,internships practical training opportunities,ms_artificial_intelligence_internships_practic...
...,...,...,...,...,...
89,Who is Marissa Barrera?,"""Marissa Barrera is a faculty member of the sp...",marissa barrera,,marissa_barrera
90,Who is Andrew Christler?,"""Andrew Christler is a faculty member of the s...",andrew christler,,andrew_christler
91,Who is Troy Clifford Dargin?,"""Troy Clifford Dargin is a faculty member of t...",troy clifford dargin,,troy_clifford_dargin
92,Who is Michaela Medved?,"""Michaela Medved is a faculty member of the sp...",michaela medved,,michaela_medved
